In [1]:
# extract package
import sqlite3
import anki
from anki.collection import Collection
import re
import os
import zipfile
import termcolor
import tqdm
import csv
import json
import shutil
import pickle
# import pandas as pd
import editdistance

EDICT_PATH = '/Users/AlexG/Documents/GitHub/mindmaps/anki_tools/data/ecdict.csv'
MEDIA_FOLDER = '/Users/AlexG/Library/Application Support/Anki2/User 1/collection.media'
PKG_PATH = '/Users/AlexG/Downloads/MyPaperNotes.apkg'
PKG_PATH = '/Users/AlexG/Downloads/ivl10.apkg'
PKG_PATH = '/Users/AlexG/Downloads/极品GRE红宝书.apkg'
DATA_PATH = '/Users/AlexG/Documents/GitHub/mindmaps/anki_tools/data'
outdir = PKG_PATH + '-decompress'
# outdir = '/Users/AlexG/Downloads/ivl10-decompress'
# ANKI2_PATH = '/Users/AlexG/Downloads/ivl10-decompress/collection.anki2'
ANKI21_PATH = outdir + '/collection.anki21'
# EXPORT_PATH_WEBSITE = '/Users/AlexG/Documents/GitHub/typingpractices/website/onepager/onepager/templates/onepager/cards.html'
EXPORT_PATH_WEBSITE = '/Users/AlexG/Documents/GitHub/anki_html/docs/gre_cards.html'

def unzip_anki(pkg_file_path, outdir):
    # Create the extract directory if it doesn't exist
    os.makedirs(outdir, exist_ok=True)

    # Unzip the .apkg file
    with zipfile.ZipFile(PKG_PATH, 'r') as zip_ref:
        zip_ref.extractall(outdir)

    print(f"Extracted files to {outdir}")

unzip_anki(PKG_PATH, outdir)
col = Collection(ANKI21_PATH)
# Fetch all decks in the collection
decks = col.decks.all()
print("Decks in collection:")
for deck in decks:
    print(f"Deck ID: {deck['id']}, Name: {deck['name']}")
# query = 'prop:ivl<=30'
cards = col.find_cards('')


Extracted files to /Users/AlexG/Downloads/极品GRE红宝书.apkg-decompress
Decks in collection:
Deck ID: 1685695741424, Name: 极品GRE红宝书
Deck ID: 1, Name: Default


In [10]:
import time
def get_lastest_review_time(col, cards):
    rev_times = []
    for card in tqdm.tqdm(cards):
        timestamp_ms = col.db.scalar(
            "SELECT max(id) FROM revlog WHERE cid = {} AND type IN (1, 3)".format(card)
        )
        rev_times.append(timestamp_ms)
    mt = max(rev_times)
    return time.localtime(mt/1000)

len(cards)
cards[0:10]
rev_times = []
for card in tqdm.tqdm(cards):
    timestamp_ms = col.db.scalar(
        "SELECT max(id) FROM revlog WHERE cid = {} AND type IN (1, 3)".format(card)
    )
    rev_times.append(timestamp_ms)
    # print('last review time: {}'.format(time.localtime(timestamp_ms/1000)))
mt = max(rev_times)
time.localtime(mt/1000)

100%|██████████| 7513/7513 [00:02<00:00, 2862.80it/s]


time.struct_time(tm_year=2025, tm_mon=2, tm_mday=5, tm_hour=20, tm_min=54, tm_sec=8, tm_wday=2, tm_yday=36, tm_isdst=0)

In [11]:
col.close()

In [ ]:

# top 3 cards by ivl
data = []
for cid in tqdm.tqdm(cards):
    card = col.get_card(cid)
    note = card.note()
    timestamp_ms = col.db.scalar(
        "SELECT max(id) FROM revlog WHERE cid = ? AND type IN (1, 3)",
        (1,)
    )
    print(f"Timestamp: {timestamp_ms}")
    import pdb
    pdb.set_trace()
    fields = note.fields
    data.append((card.ivl, fields))
data.sort()
for ivl, c in data[:3]:
    print(termcolor.colored('[ivl={}]'.format(ivl), 'red'), 
            termcolor.colored('{}'.format(c), 'green'))

col.close()